Input: Octo-Tiger HPX output, including Performance counter values

Output: CSV
```csv
iteration,locality,subgrids,amr_bound,subgrid_leaves,idle_rate_0,idle_rate_1,...,idle_rate_19
```

In [1]:
import os
import lzma
import glob
import re
import pandas as pd

In [2]:
def get_pfx_counter_line_pattern():
    return re.compile(
        r'^/[^{]+\{[^}]+\}/([^,]+,){4,5}[^,\n]+$', re.MULTILINE)


def test_pfx_counter_line_pattern(pattern):
    def test_case(subject):
        return [i.group() for i in pattern.finditer(subject)]

    subject = '''
/octotiger{locality#0/total}/subgrid_leaves,2,3602.438779,[s],32428
/octotiger{locality#1/total}/subgrid_leaves,2,3602.428582,[s],34118
/octotiger{locality#2/total}/subgrid_leaves,2,3602.430246,[s],33918
/octotiger{locality#3/total}/subgrid_leaves,2,3602.431576,[s],34443
/octotiger{locality#4/total}/subgrid_leaves,2,3602.436175,[s],33173
/threads{locality#59/total/total}/count/cumulative,44,154828.198473,[s],2.3065e+09
/threads{locality#60/total/total}/count/cumulative,44,154828.221342,[s],2.24724e+09
/threads{locality#61/total/total}/count/cumulative,44,154828.184221,[s],2.28135e+09
/threads{locality#62/total/total}/count/cumulative,44,154828.221351,[s],2.20491e+09
/threads{locality#63/total/total}/count/cumulative,44,154828.216028,[s],2.05324e+09
/threads{locality#0/pool#default/worker-thread#0}/count/cumulative,44,154828.257432,[s],1.14378e+08
/threads{locality#0/pool#default/worker-thread#1}/count/cumulative,44,154828.260203,[s],1.05219e+08
/threads{locality#0/pool#default/worker-thread#2}/count/cumulative,44,154828.262954,[s],1.04616e+08
/threads{locality#0/pool#default/worker-thread#3}/count/cumulative,44,154828.262962,[s],1.04786e+08
    '''
    expected = [
        '/octotiger{locality#0/total}/subgrid_leaves,2,3602.438779,[s],32428',
        '/octotiger{locality#1/total}/subgrid_leaves,2,3602.428582,[s],34118',
        '/octotiger{locality#2/total}/subgrid_leaves,2,3602.430246,[s],33918',
        '/octotiger{locality#3/total}/subgrid_leaves,2,3602.431576,[s],34443',
        '/octotiger{locality#4/total}/subgrid_leaves,2,3602.436175,[s],33173',
        '/threads{locality#59/total/total}/count/cumulative,44,154828.198473,[s],2.3065e+09',
        '/threads{locality#60/total/total}/count/cumulative,44,154828.221342,[s],2.24724e+09',
        '/threads{locality#61/total/total}/count/cumulative,44,154828.184221,[s],2.28135e+09',
        '/threads{locality#62/total/total}/count/cumulative,44,154828.221351,[s],2.20491e+09',
        '/threads{locality#63/total/total}/count/cumulative,44,154828.216028,[s],2.05324e+09',
        '/threads{locality#0/pool#default/worker-thread#0}/count/cumulative,44,154828.257432,[s],1.14378e+08',
        '/threads{locality#0/pool#default/worker-thread#1}/count/cumulative,44,154828.260203,[s],1.05219e+08',
        '/threads{locality#0/pool#default/worker-thread#2}/count/cumulative,44,154828.262954,[s],1.04616e+08',
        '/threads{locality#0/pool#default/worker-thread#3}/count/cumulative,44,154828.262962,[s],1.04786e+08'
    ]

    assert test_case(subject) == expected

# Counter line search and counter name parsing regex patterns
pfx_counter_line_pattern = get_pfx_counter_line_pattern()
test_pfx_counter_line_pattern(pfx_counter_line_pattern)

In [3]:
def get_general_counter_form_pattern():
    return re.compile(
        r'/(?P<object>[^{]+)\{locality#(?P<locality>\d+)/(?:(?:(?P<instance1>pool#[^/]+/[^#]+)#(?P<thread_id>\d+))|(?P<instance2>[^}]+))\}/(?P<counter>[^@]+)(?:@(?P<params>.+))?')


def test_general_counter_form_pattern(pattern):
    def run_test_case(subject):
        m = pattern.match(subject)
        assert m is not None
        return m.groupdict()

    assert run_test_case('/threads{locality#0/pool#default/worker-thread#0}/count/cumulative') == {
        'object': 'threads',
        'locality': '0',
        'instance1': 'pool#default/worker-thread',
        'thread_id': '0',
        'instance2': None,
        'counter': 'count/cumulative',
        'params': None}
    assert run_test_case('/threads{locality#61/total/total}/count/cumulative,44,154828.184221,[s],2.28135e+09') == {
        'object': 'threads',
        'locality': '61',
        'instance1': None,
        'thread_id': None,
        'instance2': 'total/total',
        'counter': 'count/cumulative,44,154828.184221,[s],2.28135e+09',
        'params': None}

general_counter_form_pattern = get_general_counter_form_pattern()
test_general_counter_form_pattern(general_counter_form_pattern)

In [4]:
hpx_output_files = glob.glob('*.txt.xz')
assert isinstance(hpx_output_files, list)
assert len(hpx_output_files) >= 1

#### Read files

In [5]:
def read_file(filepath):
    assert os.access(filepath, os.R_OK)

    # Read one file for testing
    with lzma.open(filepath, 'rt', encoding='utf-8') as hpx_output_handle:
        hpx_output = hpx_output_handle.read()
    return hpx_output


#### Read one file for testing

In [6]:
r0f = os.path.join(os.curdir, hpx_output_files[0])
print('subject:', r0f)

subject: ./64_4.txt.xz


In [7]:
hpx_output = read_file(r0f)

In [8]:
def extract_counters_df(hpx_output):
    assert isinstance(hpx_output, str)

    all_counters = []
    for pfx_counter_line in pfx_counter_line_pattern.finditer(hpx_output):
        raw_line = pfx_counter_line.group(0)
        line_split = raw_line.split(',')

        def justify_fields(line_split):
            if len(line_split) == 5:
                # Unit for count values
                line_split += ('1')
            assert len(line_split) == 6
        justify_fields(line_split)

        raw_general_name = line_split[0]
        split_general_form = general_counter_form_pattern.match(
            raw_general_name)
        assert split_general_form is not None

        countername_groups = split_general_form.groupdict()

        def unify_instance_field(countername_groups):
            if countername_groups['instance1'] is not None:
                countername_groups['instance'] = countername_groups['instance1']
            else:
                countername_groups['instance'] = countername_groups['instance2']

            del countername_groups['instance1']
            del countername_groups['instance2']
        unify_instance_field(countername_groups)

        all_counters += [(
            countername_groups["object"],
            countername_groups["locality"],
            countername_groups["instance"],
            countername_groups["counter"],
            countername_groups["thread_id"],
            countername_groups["params"],
        ) + tuple(line_split)]

    df = pd.DataFrame(all_counters, columns=[
        'objectname', 'locality', 'instance', 'countername', 'thread_id',
        'parameters', 'general_form', 'iteration', 'timestamp',
        'timestamp_unit', 'value', 'unit'])
    assert isinstance(df, pd.DataFrame)
    assert len(df) != 0
    return df

In [9]:
%%time
df = extract_counters_df(hpx_output)

CPU times: user 2.75 s, sys: 139 ms, total: 2.88 s
Wall time: 2.9 s


In [10]:
def check_and_prune_fields(df):
    assert isinstance(df, pd.DataFrame)

    def fix_units(df):
        df.locality = pd.to_numeric(df.locality, downcast='unsigned')
        df.iteration = pd.to_numeric(df.iteration, downcast='unsigned')
        df.timestamp = pd.to_numeric(df.timestamp)
        df.value = pd.to_numeric(df.value)
        df.thread_id = pd.to_numeric(df.thread_id, downcast='unsigned')
        df.loc[df.countername == 'idle-rate', 'value'] *= 0.01
    fix_units(df)

    def drop_irrelevant_counters(df):
        # Drop AGAS results
        df.drop(df.index[df.objectname == 'agas'], inplace=True)
        # Drop threads...pool#default/worker-thread...count/cumulative-phases
        df.drop(df.index[df.countername == 'count/cumulative-phases'], inplace=True)
        df.drop(df.index[df.countername == 'count/cumulative'], inplace=True)

    drop_irrelevant_counters(df)

    # Units can only be [0.01%], 1, [s], and [ns]
    def check_all_data_units(df):
        x = df.loc[(df.unit != '1')]
        x = x.loc[x.unit != '[0.01%]']
        x = x.loc[x.unit != '[s]']
        x = x.loc[x.unit != '[ns]']
        assert len(x) == 0

    check_all_data_units(df)

    def remove_unused_columns(df):
        # No parameters are expected
        assert len(df.loc[~df.parameters.isnull()]) == 0
        del df['parameters']

        del df['timestamp_unit']
        del df['unit']
        del df['timestamp']
        del df['general_form']
    remove_unused_columns(df)

In [11]:
check_and_prune_fields(df)

In [12]:
def process_df(df):
    def get_octotiger_counters(df):
        octo_pivot = df.pivot_table(
            index=[
                'iteration',
                'locality'],
            columns=['countername'],
            values='value',
            dropna=False)
        del octo_pivot['idle-rate']
        return octo_pivot
    octotiger_counters = get_octotiger_counters(df)

    def get_idle_rate_counters(df):
        idle_rate_pivot = df.pivot_table(
            index=[
                'iteration',
                'locality'],
            columns=['thread_id'],
            values='value')
        return idle_rate_pivot
    idle_rates = get_idle_rate_counters(df)

    result = pd.concat([octotiger_counters, idle_rates], axis=1)
    result.reset_index(inplace=True)
    return result

In [13]:
df = process_df(df)

In [14]:
def get_csv_output_path(original_path):
    return os.path.splitext(original_path)[0] + '.csv'
of = get_csv_output_path(r0f)

In [15]:
df.to_csv('test.csv')